### Building ANN for churn modelling

In [9]:
#import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#### Data Preprocessing

In [15]:
#import the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [18]:
#encoding categorical data| fill NA values
#label encode the gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [21]:
#one hot encode the geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [23]:
#split dataset into test and training set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [26]:
#feature scaling
#sc.fit_transform get mean and standard deviation of the training set
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [27]:
#Initialize ANN
ann = tf.keras.models.Sequential()

In [28]:
#Add input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#add second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [29]:
#add output layer (activation for non-binary is softmax) 

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#### Training the ANN

In [30]:
#compiling the ANN (for non-binary classification loss is categorical_crossentropy)

ann.compile(optimizer='adam' ,loss='binary_crossentropy' , metrics=['accuracy'])

In [31]:
#Train the ANN
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 994us/step - loss: 0.5384 - accuracy: 0.7931
Epoch 2/100
250/250 [==============================] - 0s 862us/step - loss: 0.4639 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 827us/step - loss: 0.4416 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 894us/step - loss: 0.4317 - accuracy: 0.7994
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8102
Epoch 6/100
250/250 [==============================] - 0s 759us/step - loss: 0.4194 - accuracy: 0.8181
Epoch 7/100
250/250 [==============================] - 0s 720us/step - loss: 0.4141 - accuracy: 0.8224
Epoch 8/100
250/250 [==============================] - 0s 686us/step - loss: 0.4091 - accuracy: 0.8238
Epoch 9/100
250/250 [==============================] - 0s 934us/step - loss: 0.4050 - accuracy: 0.8264
Epoch 10/100
250/250 [==============================] - 0s 678us/step - los

#### Making predictions and evaluating the model

In [33]:
# Q: is the customer going to leave the bank?
# make prediction given the following info: geo=france, credit score=600, 
# gender=male, age = 40, tenure=3, bal=60 000, products=2, credit card=yes, customer active=yes, salary=50 000

## any input to the predict method must be a 2d array (double square brackets)
#always scale data before passing to predict
#for scaling we dont need fit_transform because it will cause information leakage, its only applied during training

print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


##### Predict the test results

In [35]:
#Predicting the test results

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making the confusion matrix

In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1521   74]
 [ 197  208]]


0.8645